In [2]:
import numpy as np
import warnings
import pandas as pd
import requests
import urllib3

data = pd. read_csv("./Data/eq2015_cleaned.csv")

district_id_list = np.sort(data["district_id"].value_counts().index)

url_1 = "https://eq2015.npc.gov.np/api/distributions?location_code=" # https://eq2015.npc.gov.np/api/distributions?location_code=1&themes=indv_migration&depth=2
url_2 = "&themes=indv_migration&depth=2"

data['district_name'] = [0] * len(data)
data['vdcmun_name'] = [0] * len(data)
data["present_grade"] = [0] * len(data)

not_in_url = [] 

present_grade_list = [] 

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    for i in range(len(district_id_list)) :
        url = url_1 + str(district_id_list[i]) + url_2

        response = requests.get(url, verify=False)
    
        if response.status_code == 200:
            # JSON 데이터 파싱
            region = response.json()
            
            ward_id_list = list(data[(data["district_id"] == district_id_list[i])]["ward_id"].value_counts().index)
            
            for j in range(len(region["data"]['indv_migration'])) :
                district_id = int(region["data"]['indv_migration'][j]["district_id"])
                vdcmun_id = int(region["data"]['indv_migration'][j]["vdcmun_id"])
                ward_id = int(region["data"]['indv_migration'][j]["ward_id"])
                
                district_name = region["data"]['indv_migration'][j]["district_name"]
                vdcmun_name = region["data"]['indv_migration'][j]["vdcmun_name"]
                
                value = region["data"]['indv_migration'][j]["value"]
                present_grade_list.append(value)
                
                if ward_id in ward_id_list :
                    condition = (data["district_id"] == district_id) & (data["vdcmun_id"] == vdcmun_id) & (data["ward_id"] == ward_id)
                    # 각 건물에 district_name, vdcmun_name 지정
                    
                    indices = data[condition].index
                    if len(indices) <= 0 :
                        print(f"No matching indices for condition: {district_id}, {vdcmun_id}, {ward_id}")
                    
                    data.loc[condition, 'district_name'] =  district_name
                    data.loc[condition, 'vdcmun_name'] =  vdcmun_name
                    
                    # 각 건물이 위치한 ward에 얼마나 사는지 정보 
                    data.loc[condition, 'present_grade'] = value
                    
                else :
                    not_in_url.append(region["data"]['indv_migration'][j])
                    print(district_id, vdcmun_id, ward_id, district_name, vdcmun_name, value)
                
                
                
            print(f"Success fill about {district_id_list[i]}")        
        
        else:
            print(f"Failed to retrieve data. Status code: {district_id_list[i], response.status_code}")

Success fill about 12
Success fill about 20
Success fill about 21
Success fill about 22
Success fill about 23
Success fill about 24
Success fill about 28
Success fill about 29
Success fill about 30
Success fill about 31
Success fill about 36
